## Imports

In [1]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os

In [2]:
df = pd.read_csv('SoFla Neighborhoods New Construction - Bay_harbor_islands.csv')

## Image URL collector

In [3]:
# Drop last n rows of a df. In case there are sums that need clipped
# df.drop(df.tail(n).index, inplace=True)

In [4]:
df['image_links'] = ''
links = []
for url in df['story link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find_all('img')
        counter = 0
        for image in images:
            counter += 1
            if 'jpg' in image['src']:
                links.append(image['src'])
                if counter == len(images):
                    links.append('no image found')
                break
    except:
        links.append('no_image')

### Remove the two lines of code below if this isn't for bay harbor 
### map
x = links[-1]
links.append(x)
df['image_links'] = links

In [5]:
len(links)

12

## Article link formatter

In [6]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0     <a href="https://therealdeal.com/miami/2022/06...
1     <a href="https://therealdeal.com/miami/2022/05...
2     <a href="https://therealdeal.com/miami/2022/05...
3     <a href="https://therealdeal.com/miami/2022/05...
4     <a href="https://therealdeal.com/miami/2022/05...
5     <a href="https://therealdeal.com/miami/2022/05...
6     <a href="https://therealdeal.com/miami/2022/05...
7     <a href="https://therealdeal.com/miami/2022/05...
8     <a href="https://therealdeal.com/miami/2022/05...
9     <a href="https://therealdeal.com/miami/2022/01...
10    <a href="https://therealdeal.com/miami/2021/10...
11    <a href="https://therealdeal.com/miami/2021/05...
Name: description_link, dtype: object

## Google Maps API Geolocater Setup

In [7]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [8]:
df['geo_address'] = df['address'] + ' Bay Harbor Islands, FL'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [9]:
df.columns

Index(['address', 'stories', 'units', 'developer', 'status', 'project_cost',
       'land_cost', 'loan_amount', 'lender', 'description', 'story link',
       'useful_info', 'image_links', 'short_description', 'remaining_desc',
       'description_link', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [10]:
df.at[8,'image_links']=('https://therealdeal.com/miami/wp-content/uploads/2022/08/Alta-CEO-Raimundo-Onetto.jpg')

In [11]:
df

,address,stories,units,developer,status,project_cost,land_cost,loan_amount,lender,description,...,image_links,short_description,remaining_desc,description_link,geo_address,loc,point,lat,lon,altitude
0,1135 103rd Street,8,41,Colombo's CMC Group and Morabito Properties,under construction,NaN,NaN,64500000.0,Bank OZK,"Designed by Arquitectonica, Onda Residences wi...",...,https://s13.therealdeal.com/trd/m/up/2022/06/M...,Designed by,"Arquitectonica, Onda Residences will have mor...","<a href=""https://therealdeal.com/miami/2022/06...","1135 103rd Street Bay Harbor Islands, FL","(1135 103rd St, Bay Harbor Islands, FL 33154, ...","(25.8940188, -80.13218239999999, 0.0)",25.894019,-80.132182,0.0
1,"9927, 9955 East Bay Harbor Drive",unknown,unknown,Regency Development Group,NaN,NaN,13500000.0,5000000.0,Park Ridge Community Bank,Chicago developers Igor Michin and Alex Troyan...,...,https://s12.therealdeal.com/trd/m/up/2022/05/m...,Chicago developers,Igor Michin and Alex Troyanovsky didn't retur...,"<a href=""https://therealdeal.com/miami/2022/05...","9927, 9955 East Bay Harbor Drive Bay Harbor Is...","(9955 E Bay Harbor Dr, Bay Harbor Islands, FL ...","(25.8901912, -80.13062529999999, 0.0)",25.890191,-80.130625,0.0
2,9781 East Bay Harbor Drive,unknown,unknown,Regency Development Group,NaN,NaN,4500000.0,NaN,NaN,Chicago developers Igor Michin and Alex Troyan...,...,https://s12.therealdeal.com/trd/m/up/2022/05/m...,Chicago developers,Igor Michin and Alex Troyanovsky didn't retur...,"<a href=""https://therealdeal.com/miami/2022/05...","9781 East Bay Harbor Drive Bay Harbor Islands, FL","(9781 E Bay Harbor Dr, Bay Harbor Islands, FL ...","(25.8888104, -80.12955529999999, 0.0)",25.888810,-80.129555,0.0
3,10301 East Bay Harbor Drive,unknown,unknown,Regency Development Group,NaN,NaN,4500000.0,NaN,NaN,Chicago developers Igor Michin and Alex Troyan...,...,https://s12.therealdeal.com/trd/m/up/2022/05/m...,Chicago developers,Igor Michin and Alex Troyanovsky didn't retur...,"<a href=""https://therealdeal.com/miami/2022/05...",10301 East Bay Harbor Drive Bay Harbor Islands...,"(10301 E Bay Harbor Dr, Bay Harbor Islands, FL...","(25.8937949, -80.1319118, 0.0)",25.893795,-80.131912,0.0
4,9110 West Bay Harbor Drive,8,10,Menachem Kranz,planned,NaN,6500000.0,NaN,MidFirst Bank,"Kranz said he plans to build an eight-story, 1...",...,https://s13.therealdeal.com/trd/m/up/2022/05/m...,Kranz said he plans,"to build an eight-story, 10-unit Kobi Karp-de...","<a href=""https://therealdeal.com/miami/2022/05...","9110 West Bay Harbor Drive Bay Harbor Islands, FL","(9110 W Bay Harbor Dr, Bay Harbor Islands, FL ...","(25.8816755, -80.1308639, 0.0)",25.881676,-80.130864,0.0
5,10200 and 10290 East Bay Harbor Drive,NaN,77,Clara Homes,planned,NaN,32300000.0,NaN,NaN,"Clara Homes plans to develop Clara Bay Harbor,...",...,https://s14.therealdeal.com/trd/m/up/2022/05/M...,Clara Homes plans to,"develop Clara Bay Harbor, with three six-stor...","<a href=""https://therealdeal.com/miami/2022/05...",10200 and 10290 East Bay Harbor Drive Bay Harb...,"(10290 E Bay Harbor Dr, Bay Harbor Islands, FL...","(25.8933321, -80.1320806, 0.0)",25.893332,-80.132081,0.0
6,10281 West Bay Harbor Drive,5,28,Clara Homes,planned,NaN,32300000.0,NaN,NaN,"Clara Homes plans to develop Clara Bay Harbor,...",...,https://s14.therealdeal.com/trd/m/up/2022/05/M...,Clara Homes plans to,"develop Clara Bay Harbor, with three six-stor...","<a href=""https://therealdeal.com/miami/2022/05...",10281 West Bay Harbor Drive Bay Harbor Islands...,"(10281 W Bay Harbor Dr, Bay Harbor Islands, FL...","(25.8930904, -80.1333066, 0.0)",25.893090,-80.133307,0.0
7,1147-1163 100th Street,6,45,Clara Homes,planned,NaN,32300000.0,NaN,NaN,"Clara Homes plans to develop Clara Bay Harbor,...",...,https://s14.therealdeal.com/trd/m/up/2022/05/M...,Clara Homes plans to,"develop Clara Bay Harbor, with three six-stor...","<a href=""https://therealdeal.com/miami/2022/05...","1147-1163 100th Street Bay Harbor Islands, FL","(1147 100th St, B

## Correction section

In [12]:
# df.at[0,'Net decline in footprint, in sf']=('Almost 200K sf')
# df.at[1,'Address(es) of space company vacated or plans to vacate, in sf']=('690 East Middlefield Road, Mountain View. Synopsys is subletting the entire building to Waymo.')

### Change value to percent value
### df['percent_change'] = pd.Series(['{0:.2f}%'.format(val) for val in df['percent_change']], index = df.index)

## HTML popup formatter

In [13]:
df.columns

Index(['address', 'stories', 'units', 'developer', 'status', 'project_cost',
       'land_cost', 'loan_amount', 'lender', 'description', 'story link',
       'useful_info', 'image_links', 'short_description', 'remaining_desc',
       'description_link', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [14]:
def popup_html(row):
    i = row
    address = df['address'].iloc[i]
    developer = df['developer'].iloc[i]
    image = df['image_links'].iloc[i]
    description = df['description_link'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Developer: <strong>{}'''.format(developer) + '''</strong><br>
    <strong>{}'''.format(description) + '''</strong><br>
    </html>
    '''
    return html


### HTML reservoir

In [15]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [16]:
df.description_link[0]

'<a href="https://therealdeal.com/miami/2022/06/15/ugo-colombos-cmc-and-morabito-land-65m-construction-loan-for-bay-harbor-condos/" target="_blank" rel="noopener noreferrer">Designed by</a> Arquitectonica, Onda Residences will have more than 300 feet of bay frontage and a 16-slip private marina. The 41-unit, eight-story condo development is scheduled to be completed in 2023, and is more than 85 percent presold, according to the developers. Bank OZK provided a $64.5 million loan in June. Fortune International group is handling sales.'

In [17]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=15)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Bay Harbor Islands Resi Projects')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [18]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [19]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [20]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Bay_Harbor_Resi_pipeline
